# Work with Oracle

In [1]:
import cx_Oracle
import pandas as pd
import numpy as np

## 6 Dimension Summary

In [2]:
conStr = 'NARUT_E01044451/v9VzR8xsuwp#tyTb@TDM-scan.dc2.true.th:1521/TDMDBPR'
conn = cx_Oracle.connect(conStr)
cur = conn.cursor()

# print(cx_Oracle.version)
# print("Client version:", cx_Oracle.clientversion())

v_week = (2023020, )


try:
    # print("Database version:", conn.version)
    print(f'\nExecute query as Week {v_week[0]}...\n')

    sqlTxt = """
        WITH W_VAR (AS_OF_WK) AS 
        ( 
            SELECT :1 AS AS_OF_WK FROM DUAL 
        ) 
        , W_TMP_PERIOD (TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, WK_DAY_START, WK_DAY_END) AS 
        ( 
            SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, MIN(TM_KEY_DAY) AS WK_DAY_START, MAX(TM_KEY_DAY) AS WK_DAY_END 
            FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK 
            WHERE TM_KEY_WK <= (SELECT AS_OF_WK FROM W_VAR) 
            GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK 
        ) 
        SELECT * 
        FROM ( 
            SELECT TM_KEY_MTH, TM_KEY_WK, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME, DIMENSION_KPI_FLAG, BG_FLAG--, METRIC_NAME_GROUP 
                , AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD 
                , CAST(SUM(ACTUAL_AGG) AS DECIMAL(18,2)) AS ACTUAL_AGG 
                , CAST(SUM(TARGET_AGG) AS DECIMAL(18,2)) AS TARGET_AGG 
                , MAX(PPN_TM) AS PPN_TM 
            FROM ( 
                SELECT TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME 
                    , CASE 	WHEN ( METRIC_CD IN ('DNEC10006', 'TNEC10002', 'TNEC20002') --NETWORK EXPERIENCE 
                                OR METRIC_CD IN ('B1R000100', 'B2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00142') --BUSINESS GROWTH 
                                OR METRIC_CD IN ('DSER21101', 'DSER22101', 'TSER21101', 'TSER22101', 'TSER23101', 'DB2S000200', 'DB1S000200', 'TB2S000200', 'TB1S000200', 'TSER13100', 'TSER14100') --CUSTOMER RETENTION & EXPERIENCE 
                                OR METRIC_CD IN ('TNSC00011', 'TNSC00013', 'TNSC00087', 'TNSC00101', 'TNSC00144', 'TNSC00020', 'TNSC00130') --NEW S-CURVE 
                                --COST & PRODUCTIVITY (Unknown ?) 
                                OR METRIC_CD IN ('DIS10007', 'DIS10011', 'DIS10001') --DIGITALIZATION 
                            ) THEN 'Y' ELSE 'N' END DIMENSION_KPI_FLAG 
                    , CASE 	WHEN METRIC_CD IN ('B1R000100', 'B2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00142') --BUSINESS GROWTH 
                            THEN 'Y' ELSE 'N' END BG_FLAG 
                    , METRIC_NAME_GROUP, PPN_TM, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_AGG, TARGET_AGG 
                FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK 
                WHERE TM_KEY_DAY IN (SELECT WK_DAY_END FROM W_TMP_PERIOD) 
                AND AREA_TYPE = 'P' 
            ) A 
            GROUP BY TM_KEY_MTH, TM_KEY_WK, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME, DIMENSION_KPI_FLAG, BG_FLAG, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD 
        ) B 
        --ORDER BY TM_KEY_MTH, TM_KEY_WK, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD 
    """
    
    cur.execute(sqlTxt, v_week)
    # cur.execute(sqlTxt, ('2023020',))
    # cur.executemany(sqlTxt, multiple parameter)

    print(f'\nDone !!!\n')

    rows = cur.fetchall()
    
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])


except cx_Oracle.DatabaseError as e:
    print(f'Error with Oracle : {e}')


cur.close()
conn.close()


Execute query as Week 2023020...


Done !!!



In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [3]:
df.head()

,TM_KEY_MTH,TM_KEY_WK,CENTER,METRIC_GRP,COMP_CD,PRODUCT_GRP,METRIC_CD,METRIC_NAME,DIMENSION_KPI_FLAG,BG_FLAG,AGG_TYPE,RR_IND,GRY_IND,UOM,PERIOD,ACTUAL_AGG,TARGET_AGG,PPN_TM
0,202303,2023009,Customer Service Experience & Retention,Experience,DTAC,All Services,DSER02401,Abandoned Call Rate : DTAC,N,N,N,0,RYG,%,M,NaN,NaN,2023-05-22 17:50:18
1,202303,2023009,Customer Service Experience & Retention,Experience,DTAC,All Services,DSER02402,Handled Call Rate : DTAC,N,N,N,0,GYR,%,M,NaN,NaN,2023-05-22 17:50:18
2,202303,2023009,Customer Service Experience & Retention,Experience,DTAC,All Services,DSER02501,Call Center CSAT IVR (Top2Boxes) : DTAC,N,N,N,0,GYR,%,M,NaN,NaN,2023-05-22 17:50:18
3,202303,2023009,Customer Service Experience & Retention,Experience,DTAC,Postpaid,DSER21101,Postpaid Complaint : DTAC,Y,N,N,0,RYG,%,M,NaN,NaN,2023-05-22 17:50:18
4,202303,2023009,Customer Service Experience & Retention,Experience,DTAC,Prepaid,DSER22101,Prepaid Complaint : DTAC,Y,N,N,0,RYG,%,M,NaN,NaN,2023-05-22 17:50:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6092,202305,2023020,Sales & Revenue,Subs,TRUE,TVS,TB4S000500,TVS Active Subs,N,N,N,0,GYR,subs,M,0.0,1439428.65,2023-05-22 17:50:18
6093,202305,2023020,Sales & Revenue,Subs,TRUE,TVS,TB4S000501,TVS Premium,N,N,N,0,GYR,subs,M,NaN,NaN,2023-05-22 17:50:18
6094,202305,2023020,Sales & Revenue,Subs,TRUE,TVS,TB4S000502,TVS Mid,N,N,N,0,GYR,subs,M,NaN,NaN,2023-05-22 17:50:18
6095,202305,2023020,Sales & Revenue,Subs,TRUE,TVS,TB4S000503,TVS Mass,N,N,N,0,GYR,subs,M,NaN,NaN,2023-05-22 17:50:18


## Generate Output file

In [ ]:
# C:\Users\Narut4\coding\Jupyter\Work with Oracle.ipynb

# df.to_csv('Raw-Agg_Performance.csv', index=False, encoding='utf-8')